In [1]:

from datetime import datetime

import torch

from datasets.CSVDataset import CSVDataset
from train import kfold_grid_search
from utils import reduce_dataset

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

batch_size = 128
dataset = CSVDataset("data/features/ResNet50_features.csv")
dataset = reduce_dataset(dataset, discard_ratio=0.9)

# dataset, _ = generate_balanced_dataset("features/ResNet50_features.csv", discard_ratio=0.9, test_ratio=0,
#                                        undersample=False)

print(f"Dataset: {len(dataset):,} examples")

Device: cuda:0
data/features/ResNet50_features.pickle
Dataset: 39,492 examples


In [2]:

print(f"Training starts {datetime.now().isoformat()}")

Training starts 2024-09-02T17:04:37.879476


In [3]:
kfold_grid_search(dataset, device,
                  checkpoint_file_path="grid-search-33.json",
                  batch_size=batch_size,
                  hidden_layer_combs=[1, 2, 3, 4],
                  neuron_combs=[2048, 4096],
                  learning_rate_combs=[0.001, 0.0001],
                  dropout_combs=[0.2, 0.4],
                  threshold_combs=[0.5, 0.6, 0.7])

(hidden_layers=1, neurons=2048, dropout=0.2, threshold=0.5, learning_rate=0.001, weight_decay=0.0)


Testing: 100%|██████████| 62/62 [00:00<00:00, 793.56it/s]


{'test_loss': 0.26800710674255124, 'test_accuracy': 0.5916911455233482, 'test_precision': 0.2811933256036078, 'test_recall': 0.5916911455233482, 'test_f1': 0.2882509290895116, 'test_mcc': 0.0}
(hidden_layers=1, neurons=2048, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0)


Testing: 100%|██████████| 62/62 [00:00<00:00, 992.51it/s]


{'test_loss': 0.29495961047228303, 'test_accuracy': 0.5836993203663231, 'test_precision': 0.2812583513037142, 'test_recall': 0.5836993203663231, 'test_f1': 0.29659401569495036, 'test_mcc': 0.0}
(hidden_layers=1, neurons=2048, dropout=0.2, threshold=0.6, learning_rate=0.001, weight_decay=0.0)


...

{'test_loss': 0.2432899684074425, 'test_accuracy': 0.5915100169689248, 'test_precision': 0.2809537102682264, 'test_recall': 0.5915100169689248, 'test_f1': 0.28853037105956564, 'test_mcc': 0.0}
(hidden_layers=2, neurons=2048, dropout=0.4, threshold=0.6, learning_rate=0.0001, weight_decay=0.0)


Testing: 100%|██████████| 62/62 [00:00<00:00, 566.73it/s]


{'test_loss': 0.2709563552191661, 'test_accuracy': 0.5967280577914814, 'test_precision': 0.2810739087301587, 'test_recall': 0.5967280577914814, 'test_f1': 0.2825365792842999, 'test_mcc': 0.0}
(hidden_layers=2, neurons=2048, dropout=0.4, threshold=0.7, learning_rate=0.001, weight_decay=0.0)


Fold 2:  75%|███████▌  | 15/20 [00:04<00:01,  3.12it/s]


KeyboardInterrupt: 

In [4]:
print(f"Training ends {datetime.now().isoformat()}")
# for metric in ["accuracy", "precision", "recall", "f1", "mcc"]:
#     if len(model_metrics[f"test_{metric}"]) > 0:
#         print(f"Test {metric}:", model_metrics[f"test_{metric}"][-1])
# 
# plot_model_metrics(model_metrics)

Training ends 2024-09-02T03:00:05.346599
